In [95]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from pathlib import Path
from keras.preprocessing import image
import cv2

plt.style.use('seaborn')

In [96]:
xtrain=[]
y=pd.read_csv('Datasets/pokemon/Train/train.csv').values
ytrain=y[:,-1]
image_ids=y[:,0]

print(image_ids[:3])

for i in image_ids:
    img=image.load_img('Datasets/pokemon/Train/Images/'+i,target_size=(32,32))
    img=image.img_to_array(img)
    xtrain.append(img)

['296.jpg' '297.jpg' '298.jpg']


/Users/harshit/opt/anaconda3/lib/python3.7/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [97]:
xtrain=np.array(xtrain,dtype='float32')/255
print(xtrain.shape,ytrain.shape)
xtrain=xtrain.reshape((xtrain.shape[0],-1))
ytrain=np.array(ytrain)
print(xtrain.shape,ytrain.shape)

# ytrain[ytrain=='Pikachu']=0
# ytrain[ytrain=='Charmander']=1
# ytrain[ytrain=='Bulbasaur']=2

(304, 32, 32, 3) (304,)
(304, 3072) (304,)


In [98]:
import random
combined=list(zip(xtrain,ytrain))
random.shuffle(combined)
xtrain,ytrain=zip(*combined)

### Grid Search

In [99]:
import multiprocessing
cpus=multiprocessing.cpu_count()
print(cpus)

4


In [100]:
params=[
    {
        'kernel':['linear','rbf','poly','sigmoid'],
        'C':[0.1,0.2,0.5,0.8,1.0,1.2,1.5,2.0,5.0]
    }
]
#c is the penalty constant

In [101]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
gs = GridSearchCV(estimator=svm.SVC(),param_grid=params,scoring="accuracy",cv=5,n_jobs = cpus)
#to speed up process no of cpus can be given as a parameter for parallel running

In [102]:
gs.fit(xtrain,ytrain)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=4,
             param_grid=[{'C': [0.1, 0.2, 0.5, 0.8, 1.0, 1.2, 1.5, 2.0, 5.0],
                          'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [103]:
print(gs.best_estimator_,gs.best_score_)

SVC(C=1.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 0.8944808743169398


In [104]:
classifier=gs.best_estimator_

### Testing on test data

In [143]:
test = pd.read_csv('Datasets/pokemon/Test/test.csv')
xtest=[]
test_image_ids = np.array(test).reshape(-1,)
for i in test_image_ids:
    img=image.load_img('Datasets/pokemon/Test/Images/'+i,target_size=(32,32))
    img=image.img_to_array(img)
    xtest.append(img)

In [144]:
xtest=np.array(xtest,dtype='float32')/255
xtest=xtest.reshape((xtest.shape[0],-1))
print(xtest.shape)

test_image_id=np.array(test_image_id)
print(test_image_id.shape)

(123, 3072)
(123, 1)


In [145]:
predictions=gs.predict(xtest)

In [146]:
print(predictions.shape,predictions)

(123,) ['Bulbasaur' 'Charmander' 'Bulbasaur' 'Charmander' 'Pikachu' 'Charmander'
 'Pikachu' 'Bulbasaur' 'Bulbasaur' 'Charmander' 'Bulbasaur' 'Bulbasaur'
 'Bulbasaur' 'Bulbasaur' 'Charmander' 'Bulbasaur' 'Charmander'
 'Charmander' 'Pikachu' 'Charmander' 'Pikachu' 'Bulbasaur' 'Charmander'
 'Bulbasaur' 'Charmander' 'Charmander' 'Bulbasaur' 'Bulbasaur' 'Pikachu'
 'Pikachu' 'Pikachu' 'Pikachu' 'Charmander' 'Bulbasaur' 'Pikachu'
 'Bulbasaur' 'Bulbasaur' 'Bulbasaur' 'Charmander' 'Bulbasaur' 'Bulbasaur'
 'Charmander' 'Pikachu' 'Charmander' 'Charmander' 'Bulbasaur' 'Charmander'
 'Bulbasaur' 'Charmander' 'Charmander' 'Pikachu' 'Charmander' 'Pikachu'
 'Bulbasaur' 'Charmander' 'Bulbasaur' 'Charmander' 'Pikachu' 'Pikachu'
 'Charmander' 'Charmander' 'Pikachu' 'Charmander' 'Charmander' 'Pikachu'
 'Pikachu' 'Bulbasaur' 'Pikachu' 'Pikachu' 'Bulbasaur' 'Charmander'
 'Bulbasaur' 'Bulbasaur' 'Pikachu' 'Charmander' 'Charmander' 'Charmander'
 'Charmander' 'Pikachu' 'Bulbasaur' 'Pikachu' 'Bulbasaur' 'Charman

In [147]:
mydict = {}
mydict['ImageId']=test_image_ids
mydict['NameOfPokemon']=predictions

In [151]:
df=pd.DataFrame(mydict)
# print(df.values)

In [152]:
df.to_csv('Pokemon_challenge_sol.csv',index=False)